In [86]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import warnings
from tqdm import tqdm
import seaborn as sns
random_state =537

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 100)

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

import gc
gc.collect()

343480

In [89]:
train.LABEL.value_counts()

LABEL
UA      832770
HU14     10754
HU07      3728
HU06      3178
HU19       759
HU12       676
HU11       439
HU15       415
Name: count, dtype: int64

In [87]:
test = pd.read_csv("/kaggle/input/anadolu-hayat-emeklilik-datathon-2024/test.csv")
train = pd.read_csv("/kaggle/input/anadolu-hayat-emeklilik-datathon-2024/train.csv")
sub = pd.read_csv("/kaggle/input/anadolu-hayat-emeklilik-datathon-2024/sample_submission.csv")
test2 = test.copy()


In [29]:
label = "LABEL"
drop = ["MUSTERI_ID"]

product_feat_cols = list(train.iloc[:,-42:-1].columns)
soru_cols = [col for col in train.columns if col.startswith("SORU") ]
finans_cols = list(train.drop(product_feat_cols+soru_cols,axis=1).select_dtypes(np.number).iloc[:,5:].columns)
hesap_cols = list(train.drop([label]+product_feat_cols+soru_cols+finans_cols+drop,axis=1).columns)

# product_feat_cols

In [30]:
train[product_feat_cols]

,BU01,BU02,BU03,BU04,BU05,BU06,BU07,BU08,BU09,BU10,BU11,BU12,BU13,BU14,BU15,BU16,BU17,BU18,BU19,BU20,BU21,BU22,BU23,BU24,HU01,HU02,HU03,HU04,HU05,HU06,HU07,HU10,HU11,HU12,HU13,HU14,HU15,HU16,HU17,HU18,HU19
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852714,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
852715,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
852716,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
852717,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [31]:
xls = pd.ExcelFile('/kaggle/input/anadolu-hayat-emeklilik-datathon-2024/Urun Ozellikleri.xlsx')
urun = pd.read_excel(xls, 'Sayfa1')
urun.drop([31,32],inplace=True)
new_cols = []


for a in tqdm(['Branş ', 'Grup/Ferdi', 'Kredili/Kredisiz','Biriktiren/ Koruma', 'Sigortas Süresi ']):
    for i in urun[a].unique():
        train[f"{a.lower()}_{i}"] = 0 
        test[f"{a.lower()}_{i}"] = 0 
        new_cols.append(f"{a.lower()}_{i}")
        for ii in urun.loc[urun[a] == i]["Ürün "].unique():
            train.loc[train[ii]!= 0,[f"{a.lower()}_{i}"]] = 1
            test.loc[test[ii]!= 0,[f"{a.lower()}_{i}"]] = 1

100%|██████████| 5/5 [00:00<00:00,  5.90it/s]


In [32]:
def prod_exc(row):
    return [col for col, val in row.items() if val != 0]

train["products_list"] = list(tqdm(train[product_feat_cols].progress_apply(prod_exc, axis=1)))
test["products_list"] = list(tqdm(test[product_feat_cols].progress_apply(prod_exc, axis=1)))
train["n_of_product"] = train.products_list.apply(lambda x:len(x))
test["n_of_product"] = test.products_list.apply(lambda x:len(x))

  0%|          | 0/852719 [00:00<?, ?it/s]

100%|██████████| 852719/852719 [00:00<00:00, 3414008.13it/s]


  0%|          | 0/195631 [00:00<?, ?it/s]

100%|██████████| 195631/195631 [00:00<00:00, 2926106.60it/s]


In [33]:
product_feat_cols = product_feat_cols+["products_list","n_of_product"]+new_cols

In [34]:
# for i in product_feat_cols:
#     fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
#     sizes = train[i].value_counts()
#     labels = list(train[i].value_counts().index)
    

#     axes[0].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
#     axes[0].legend(labels=labels,bbox_to_anchor=(0,1),shadow=True,numpoints=1)
#     axes[0].set_title("Train")
   
#     sizes_test = test[i].value_counts()
#     labels_test = test[i].value_counts().index
    
#     axes[1].pie(sizes_test, labels=labels_test, autopct='%1.1f%%', startangle=90)
#     axes[1].legend(labels=labels_test,bbox_to_anchor=(0, 1),shadow=True,numpoints=1)
#     axes[1].set_title("Test")
    

#     plt.suptitle(i)
#     plt.show()

In [38]:
train[product_feat_cols]

,BU01,BU02,BU03,BU04,BU05,BU06,BU07,BU08,BU09,BU10,BU11,BU12,BU13,BU14,BU15,BU16,BU17,BU18,BU19,BU20,BU21,BU22,BU23,BU24,HU01,HU02,HU03,HU04,HU05,HU06,HU07,HU10,HU11,HU12,HU13,HU14,HU15,HU16,HU17,HU18,HU19,products_list,n_of_product,branş _Bireysel Emeklilik,branş _otomatik Katılım,branş _Hayat Sigortası,grup/ferdi_Ferdi,grup/ferdi_Grup,kredili/kredisiz_Krediden bağımsız,kredili/kredisiz_Kredi Bağlantılı,biriktiren/ koruma_Biriktiren,biriktiren/ koruma_Koruma,sigortas süresi _Uzun Süreli,sigortas süresi _Yıllık
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[BU01],1,1,0,0,1,0,1,0,1,0,1,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"[BU09, BU19]",2,1,0,0,1,0,1,0,1,0,1,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,"[BU09, BU17, HU12]",3,1,0,1,1,0,1,0,1,1,1,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[BU17],1,1,0,0,1,0,1,0,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[BU16],1,1,0,0,0,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
852714,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[BU17],1,1,0,0,1,0,1,0,1,0,1,0
852715,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[BU09],1,1,0,0,1,0,1,0,1,0,1,0
852716,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[BU23],1,1,0,0,1,0,1,0,1,0,1,0
852717,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,[BU02],1,1,0,0,1,0,1,0,1,0,1,0


# soru_cols

In [84]:
drop_finans=["SORU_YATIRIM_KARAKTERI_CVP",    
"SORU_YATIRIM_KARAKTERI_RG",                   
"SORU_GELIR_CVP",               
"SORU_GELIR_RG",                 
"SORU_COCUK_SAYISI_CVP",        
"SORU_COCUK_SAYISI_RG"]

In [82]:
train[soru_cols].isna().sum()/train.shape[0]*100

SORU_YATIRIM_KARAKTERI_CVP    84.438250
SORU_YATIRIM_KARAKTERI_RG     84.438250
SORU_MEDENI_HAL_CVP            0.208744
SORU_MEDENI_HAL_RG             0.208744
SORU_EGITIM_CVP               14.904676
SORU_EGITIM_RG                14.904676
SORU_GELIR_CVP                32.590103
SORU_GELIR_RG                 32.590103
SORU_COCUK_SAYISI_CVP         48.850794
SORU_COCUK_SAYISI_RG          48.850794
dtype: float64

In [61]:
# for i in train.LABEL.unique():
#     print(i,"*"*20,train.loc[train.LABEL == i].shape[0])
#     print(train.loc[train.LABEL == i,soru_cols].isna().sum()/train.loc[train.LABEL == i].shape[0])

In [83]:
test[soru_cols].isna().sum()/test.shape[0]*100

SORU_YATIRIM_KARAKTERI_CVP    86.000685
SORU_YATIRIM_KARAKTERI_RG     86.000685
SORU_MEDENI_HAL_CVP            0.161529
SORU_MEDENI_HAL_RG             0.161529
SORU_EGITIM_CVP               13.844432
SORU_EGITIM_RG                13.844432
SORU_GELIR_CVP                40.739453
SORU_GELIR_RG                 40.739453
SORU_COCUK_SAYISI_CVP         58.035281
SORU_COCUK_SAYISI_RG          58.035281
dtype: float64

In [68]:
train[soru_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852719 entries, 0 to 852718
Data columns (total 10 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   SORU_YATIRIM_KARAKTERI_CVP  132698 non-null  object 
 1   SORU_YATIRIM_KARAKTERI_RG   132698 non-null  float64
 2   SORU_MEDENI_HAL_CVP         850939 non-null  object 
 3   SORU_MEDENI_HAL_RG          850939 non-null  float64
 4   SORU_EGITIM_CVP             725624 non-null  object 
 5   SORU_EGITIM_RG              725624 non-null  float64
 6   SORU_GELIR_CVP              574817 non-null  object 
 7   SORU_GELIR_RG               574817 non-null  float64
 8   SORU_COCUK_SAYISI_CVP       436159 non-null  float64
 9   SORU_COCUK_SAYISI_RG        436159 non-null  float64
dtypes: float64(6), object(4)
memory usage: 65.1+ MB


In [69]:
train[soru_cols].describe()

,SORU_YATIRIM_KARAKTERI_RG,SORU_MEDENI_HAL_RG,SORU_EGITIM_RG,SORU_GELIR_RG,SORU_COCUK_SAYISI_CVP,SORU_COCUK_SAYISI_RG
count,132698.000000,850939.000000,725624.000000,574817.000000,436159.000000,436159.000000
mean,28.250674,87.329307,84.778107,103.253851,0.112335,141.920219
std,11.966714,69.144079,71.030740,63.343308,4.568026,51.963410
min,0.000000,0.000000,0.000000,0.000000,0.000000,72.000000
25%,20.000000,23.000000,21.000000,55.000000,0.000000,98.000000
50%,29.000000,78.000000,68.000000,97.000000,0.000000,124.000000
75%,38.000000,126.000000,125.000000,141.000000,0.000000,184.000000
max,50.000000,1482.000000,1482.000000,1482.000000,3000.000000,1482.000000


In [72]:
print(train["SORU_YATIRIM_KARAKTERI_CVP"].isna().sum())
print(train["SORU_YATIRIM_KARAKTERI_CVP"].describe())
print(train["SORU_YATIRIM_KARAKTERI_CVP"].value_counts())


720021
count       132698
unique           5
top       Temkinli
freq         35414
Name: SORU_YATIRIM_KARAKTERI_CVP, dtype: object
SORU_YATIRIM_KARAKTERI_CVP
Temkinli    35414
Cesur       33284
Dengeli     26181
Yetkin      19402
Bilge       18417
Name: count, dtype: int64


In [93]:
soru_cols = list(set(soru_cols)-set(drop_finans))

In [97]:
train[soru_cols]

,SORU_EGITIM_RG,SORU_MEDENI_HAL_RG,SORU_EGITIM_CVP,SORU_MEDENI_HAL_CVP
0,68.0,68.0,(Diğer),Single
1,11.0,11.0,İlköğretim,Married
2,235.0,235.0,Lisans,Single
3,111.0,111.0,Lise,Married
4,91.0,91.0,Lisans,Married
...,...,...,...,...
852714,9.0,9.0,Lisans,Married
852715,53.0,53.0,Lisans,Married
852716,234.0,234.0,Lise,Single
852717,74.0,74.0,(Diğer),Single


In [ ]:
#